In [ ]:
# Import libraries
import os
import sys
import subprocess
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from pyfonts import set_default_font, load_google_font
import cmcrameri.cm as cmc

# Find GRASS Python packages
sys.path.append(
  subprocess.check_output(
    ["grass", "--config", "python_path"],
    text=True
    ).strip()
  )

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj
from grass.tools import Tools
import grass.script.array as ga

# Create a temporary folder
import tempfile
temporary = tempfile.TemporaryDirectory()

# Create a project in the temporary directory
gs.create_project(path=temporary.name, name="mapalgebra")

# Start GRASS in this project
session = gj.init(Path(temporary.name, "mapalgebra"))
tools = Tools()

In [ ]:
# Set region
rows = 200
cols = 800
resolution = 1
tools.g_region(s=0, w=0, n=rows * resolution, e=cols * resolution, res=resolution)

In [ ]:
# Generate fractal terrain
tools.r_surf_fractal(output="fractal", dimension=2.25, seed=39)
tools.r_mapcalc(expression="elevation = fractal / 10", overwrite=True)
tools.r_mapcalc(expression="elevation = abs(elevation)", overwrite=True)
tools.r_mapcalc(expression="elevation = if(elevation > 30, 30, elevation)", overwrite=True)
tools.r_relief(input="elevation", output="relief", zscale=1)

# Convert to array
elevation = ga.array("elevation")

In [ ]:
# Visualize
m = gj.Map(width=800)
m.d_rast(map="elevation")
m.d_legend(raster="elevation", at=(5, 95, 1, 3))
m.show()

In [ ]:
# Set font
font = load_google_font("Fira Sans")
set_default_font(font)
plt.rcParams.update({"font.size": 6})

# Normalize colors
norm = mcolors.TwoSlopeNorm(vmin=0, vcenter=10, vmax=30)
norm(elevation)

# Plot figure
figure = plt.figure()
ax = figure.add_subplot()
ax.set_xticks([])
ax.set_yticks([])
image = ax.imshow(elevation, cmap=cmc.bukavu, norm=norm)
ticks = [0, 10, 30]
legend = figure.colorbar(image, shrink=0.275)
legend.ax.tick_params(size=0)
legend.set_ticks(ticks)

# Save image
figure.savefig(
    "figure_3.png",
    dpi=600,
    bbox_inches="tight",
    )